In [1]:
#data augmentation in python

In [1]:
#CREATE an cnn model and understand the working with keras tuner
!pip install -U keras-tuner


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 3.7 MB/s eta 0:00:00


In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np


In [3]:
print(tf.__version__)

2.15.0


In [4]:
#using fashion minst dataset
fashion_minst = keras.datasets.fashion_mnist

In [5]:
(train_images,train_labels),(test_images,test_labels) = fashion_minst.load_data()

4422102/4422102 [==============================] - 0s 0us/step


In [6]:
#scale down the images
train_imaages = train_images/255.0
test_images = test_images/255.0

In [7]:
train_images[0].shape

(28, 28)

In [8]:
train_images = train_images.reshape(len(train_images),28,28,1)
test_images = test_images.reshape(len(test_images),28,28,1)

In [9]:
from tensorflow import keras
from kerastuner import HyperParameters

def model(hp):
    model = keras.Sequential([
        keras.layers.Conv2D(
            filters=hp.Int('conv1_filter', min_value=32, max_value=128, step=16),
            kernel_size=hp.Choice('conv1_kernel', values=[3, 5]),
            activation='relu',
            input_shape=(28, 28, 1)
        ),
        keras.layers.Conv2D(
            filters=hp.Int('conv2_filter', min_value=32, max_value=64, step=16),
            kernel_size=hp.Choice('conv2_kernel', values=[3, 5]),
            activation='relu'
        ),
        keras.layers.Flatten(),
        keras.layers.Dense(
            units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
            activation='relu'
        ),
        keras.layers.Dense(10, activation='softmax')
    ])

    model.compile(
        optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    return model


<ipython-input-9-837e889aeb47>:2: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner import HyperParameters


In [10]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [11]:
tuner_search = RandomSearch(
    model,
    objective = 'val_accuracy',
    max_trials = 5,
    directory = 'output',
    project_name = 'CNN_fashion')

In [12]:
tuner_search.search(train_images,train_labels, epochs =3 , validation_split = 0.1)

Trial 5 Complete [00h 00m 42s]
val_accuracy: 0.8766666650772095

Best val_accuracy So Far: 0.8793333172798157
Total elapsed time: 00h 02m 42s


In [13]:
model = tuner_search.get_best_models(num_models = 1)[0]


In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 32)        9248      
                                                                 
 flatten (Flatten)           (None, 18432)             0         
                                                                 
 dense (Dense)               (None, 96)                1769568   
                                                                 
 dense_1 (Dense)             (None, 10)                970       
                                                                 
Total params: 1780106 (6.79 MB)
Trainable params: 1780106 (6.79 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [15]:
model.fit(train_images,train_labels,epochs=10,validation_split=0.1,initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 9s 4ms/step - loss: 0.1496 - accuracy: 0.9448 - val_loss: 0.3819 - val_accuracy: 0.8823
Epoch 5/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.1186 - accuracy: 0.9569 - val_loss: 0.4710 - val_accuracy: 0.8857
Epoch 6/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0969 - accuracy: 0.9647 - val_loss: 0.4891 - val_accuracy: 0.8833
Epoch 7/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0788 - accuracy: 0.9723 - val_loss: 0.5672 - val_accuracy: 0.8840
Epoch 8/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0665 - accuracy: 0.9774 - val_loss: 0.6204 - val_accuracy: 0.8910
Epoch 9/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0612 - accuracy: 0.9791 - val_loss: 0.7536 - val_accuracy: 0.8853
Epoch 10/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0526 - accuracy: 0.9828 - val_loss: 0.7867 - val_accuracy

In [16]:
model.evaluate(test_images,test_labels)

313/313 [==============================] - 1s 2ms/step - loss: 10.9555 - accuracy: 0.1000


[10.955467224121094, 0.10000000149011612]